In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaleido

     |████████████████████████████████| 79.9 MB 196 kB/s             


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go
import glob
import os

In [4]:
def create_dir (direct):
  if not os.path.exists(direct):
    os.mkdir(direct)

In [5]:
ROOT_DIR = "/kaggle/input/ferset" #MT Directory
DATASET_02 = ROOT_DIR + "/02_FER"
DATASET_02_TRAIN = DATASET_02 + "/train"
DATASET_02_TEST = DATASET_02+ "/test"
DATASET_02_AUG = "/kaggle/input/fer-augment-train/Aug_train"
KAGGLE_START = "/kaggle/input"

In [6]:
import PIL

image = PIL.Image.open(DATASET_02_TRAIN + "/surprise/Training_99984132.jpg")
#image to open

width, height = image.size

print(image.mode)
print(width, height)

L
48 48


In [7]:
datagen = ImageDataGenerator()
image_size = (224, 224)
colormode = 'rgb'
test_set = datagen.flow_from_directory(DATASET_02_TEST, target_size=image_size, color_mode =colormode, batch_size=32, shuffle=False)
train_set = datagen.flow_from_directory(DATASET_02_TRAIN, target_size=image_size, color_mode =colormode, batch_size=32)

Found 7178 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [8]:
def create_model_V1():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    #model.add(layers.Dropout(0.4))
    #model.add(layers.Conv2D(16, (3, 3), activation='relu'))
    #model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))
    
    return model


#model_1 = create_model_V1()
#model_1.summary()

In [9]:
def get_data_df(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [10]:
def get_data_df_2(dir):
  # modified code from: https://www.kaggle.com/namgalielei/simple-load-images-and-count-number-of-each-class

  train_df = pd.DataFrame()

  trainset = glob.glob(dir)
  #print(trainset)
  train_df['filename'] = [img.split("/")[-5]+ "/" +img.split("/")[-4] + "/" + img.split("/")[-3] + "/" + img.split("/")[-2]+"/"+img.split("/")[-1] for img in trainset]
  train_df['label'] = [img.split("/")[-2] for img in trainset]

  return train_df

In [11]:
train_df = get_data_df_2(DATASET_02_TRAIN+"/*/*.jpg")
aug_df = get_data_df_2(DATASET_02_AUG+"/*/*.jpg")
aug_df['filename'] = aug_df['filename'].str[6:]
train_df

,filename,label
0,ferset/02_FER/train/surprise/Training_66056468...,surprise
1,ferset/02_FER/train/surprise/Training_26522394...,surprise
2,ferset/02_FER/train/surprise/Training_63239185...,surprise
3,ferset/02_FER/train/surprise/Training_67725241...,surprise
4,ferset/02_FER/train/surprise/Training_71308969...,surprise
...,...,...
28704,ferset/02_FER/train/happy/Training_18866880.jpg,happy
28705,ferset/02_FER/train/happy/Training_16513509.jpg,happy
28706,ferset/02_FER/train/happy/Training_10834907.jpg,happy
28707,ferset/02_FER/train/happy/Training_30459679.jpg,happy


In [12]:
final_train = train_df.append(aug_df, sort=False, ignore_index=True)
final_train

,filename,label
0,ferset/02_FER/train/surprise/Training_66056468...,surprise
1,ferset/02_FER/train/surprise/Training_26522394...,surprise
2,ferset/02_FER/train/surprise/Training_63239185...,surprise
3,ferset/02_FER/train/surprise/Training_67725241...,surprise
4,ferset/02_FER/train/surprise/Training_71308969...,surprise
...,...,...
57413,fer-augment-train/Aug_train/happy/Clock90_Trai...,happy
57414,fer-augment-train/Aug_train/happy/CounterClock...,happy
57415,fer-augment-train/Aug_train/happy/Clock90_Trai...,happy
57416,fer-augment-train/Aug_train/happy/Clock90_Trai...,happy


In [13]:
total = 0
value_list = []
for item in os.listdir(DATASET_02_TRAIN):
    y = len(os.listdir(DATASET_02_TRAIN+'/'+item))+len(os.listdir(DATASET_02_TEST+'/'+item))
    value_list.append(y)
    total += y
    print(item)
print(total)
print(value_list)
weights = {}
for n in range(0,len(value_list)):
    w = total/(len(value_list)*value_list[n])
    weights[n] = w
    
print(weights)

surprise
fear
angry
neutral
sad
disgust
happy
35887
[4002, 5121, 4953, 6198, 6077, 547, 8989]
{0: 1.2810380524023703, 1: 1.001115853488437, 2: 1.035072539009547, 3: 0.8271562255105334, 4: 0.8436258492207152, 5: 9.372420997649517, 6: 0.5703319930708962}


In [14]:
create_dir('/kaggle/working/Saved_CNN_Models')


In [15]:
from sklearn.model_selection import KFold, StratifiedKFold

#train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
#Y = train_data[['label']]

#Y = final_train[['label']]
#n = len(final_train)   

skf = StratifiedKFold(n_splits = 5, random_state = 7, shuffle = True) 

#Un needed preprocessing aside from rescale
#idg = ImageDataGenerator(rescale=1./255)
create_dir('/kaggle/working/Saved_CNN_Models')
#model name
def get_model_name(k):
    return 'model_'+str(k)+'.h5'
def model_test(fold_count, epoch, model_method, image_size, mode):
    skf = StratifiedKFold(n_splits = fold_count, random_state = 7, shuffle = True)
    #train_data = get_data_df(DATASET_02_TRAIN+"/*/*.jpg")
    train_data = final_train
      #Incorperate preprocessing in the method 
      #####################################################################
      #aug_train = get_data_df(DATASET_01_ROOT+"/Aug_train/*/*.jpg")
      #aug_train['filename'] = aug_train['filename'].str[7:]
      #final_train = test_train.append(aug_train, sort=False, ignore_index=True)


      #######################################################################
      #idg = ImageDataGenerator(rescale=1./255)
    idg = ImageDataGenerator()
    Y = train_data[['label']]
    n = len(train_data)    
    VALIDATION_ACCURACY = []
    VALIDATION_LOSS = []

      #create a directory
    save_dir = '/kaggle/working/Saved_CNN_Models/'+model_method.__name__+'_'+str(epoch)+'epochs/'
    create_dir(save_dir)
    
    #img_dir = DATASET_02_TRAIN
    img_dir = KAGGLE_START

    fold_var = 1

      #n is number of samples, doesn't care about x values for split, just y
    for train_index, val_index in skf.split(np.zeros(n),Y):

        #Index value for fold
        training_data = train_data.iloc[train_index]
        validation_data = train_data.iloc[val_index]
        #generator for train
        #print("hi")
        #print(validation_data.head())
        #Modify the training size, have an input adjustability 
        train_data_generator = idg.flow_from_dataframe(training_data, directory = img_dir,
                                                       target_size=image_size, batch_size=32, color_mode =mode,
                                                       x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #generator for validation
        valid_data_generator  = idg.flow_from_dataframe(validation_data, directory = img_dir,
                                                        target_size=image_size, color_mode =mode,batch_size=32,
                                                        x_col = "filename", y_col = "label",shuffle = True)
        #print("hi")
        #create a method for this
        # CREATE NEW MODEL
        model = model_method()
        # COMPILE NEW MODEL
        model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])

        # CREATE CALLBACKS
        #Unsure about checpoint
        checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+model_method.__name__+"_"+get_model_name(fold_var), 
                    monitor='val_accuracy', verbose=1, 
                    save_best_only=True, mode='max')

        callbacks_list = [checkpoint]
        # There can be other callbacks, but just showing one because it involves the model name
        # This saves the best model
        # FIT THE MODEL
        #Include the weights for classes
        history = model.fit(train_data_generator,
                epochs=epoch,
                callbacks=callbacks_list,
                validation_data=valid_data_generator,
                class_weight = weights,
                verbose = 2)

        #PLOT HISTORY
        #Plots for whichever
        #		:
        #plt.plot(history.history['binary_accuracy'], label='accuracy')
        #plt.plot(history.history['val_binary_accuracy'], label = 'val_accuracy')
        #plt.xlabel('Epoch')
        #plt.ylabel('Accuracy')
        #plt.ylim([0.3, 1])
        #plt.legend(loc='lower right')
        #plt.show()
        #frame = {'Accuracy': history.history['binary_accuracy'], 'Val_Accuracy': history.history['val_binary_accuracy']}
        #fig = px.line(frame)
        #fig.show()
        fig = go.Figure()
        fig.add_trace(go.Scatter(y= history.history['accuracy'],
                          mode='lines+markers',
                          name='Training Accuracy'))
        fig.add_trace(go.Scatter(y= history.history['val_accuracy'],
                          mode='lines+markers',
                          name='Validation Accuracy'))
        fig.update_yaxes(range=[0, 1])
        fig.update_layout(
            title="Fold "+ str(fold_var)+ "'s performance",
            title_x=0.5,
            xaxis_title="Epochs",
            yaxis_title="Accuracy",
            autosize=False,
            width=800,
            height=400,
            margin=dict(l=20, r=20, t=40, b=20),)
        fig.show()
        graph_dir = '/kaggle/working/img'
        create_dir(graph_dir)
        graph_dir = graph_dir+'/'+model_method.__name__
        create_dir(graph_dir)
        fig.write_image(graph_dir+'/fold'+str(fold_var)+'.png') 

        #		:
        # LOAD BEST MODEL to evaluate the performance of the model
        model.load_weights(save_dir+model_method.__name__+"_" + "model_"+str(fold_var)+".h5")

        results = model.evaluate(test_set)
        results = dict(zip(model.metrics_names,results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_LOSS.append(results['loss'])

        tf.keras.backend.clear_session()

        fold_var += 1

In [16]:
#model_test(5, 1, create_model_V1, (48,48), 'rgb')

In [17]:
def transfer_learning_model_resnet_50():

  IMG_SHAPE = (224,224,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.resnet50.preprocess_input(inputs)
  base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(224,224,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 130
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_resnet_50 = transfer_learning_model_resnet_50()

2022-04-24 01:35:31.468127: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 01:35:31.556402: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 01:35:31.557250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-24 01:35:31.558378: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

94781440/94765736 [==============================] - 0s 0us/step
175


In [18]:
transfer_model_resnet_50.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 14343 

In [19]:
model_test(5, 30, transfer_learning_model_resnet_50, (224,224,), 'rgb')

Found 45934 validated image filenames belonging to 7 classes.
Found 11484 validated image filenames belonging to 7 classes.
175


2022-04-24 01:36:22.515967: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2022-04-24 01:36:26.630581: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1436/1436 - 303s - loss: 2.2786 - accuracy: 0.3526 - val_loss: 1.3659 - val_accuracy: 0.4855

Epoch 00001: val_accuracy improved from -inf to 0.48555, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_1.h5


...

Epoch 2/30
1436/1436 - 138s - loss: 1.7190 - accuracy: 0.4774 - val_loss: 1.3161 - val_accuracy: 0.5177

Epoch 00002: val_accuracy improved from 0.43669 to 0.51768, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 3/30
1436/1436 - 138s - loss: 1.3706 - accuracy: 0.5760 - val_loss: 1.3103 - val_accuracy: 0.5368

Epoch 00003: val_accuracy improved from 0.51768 to 0.53683, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 4/30
1436/1436 - 138s - loss: 1.0581 - accuracy: 0.6723 - val_loss: 1.3551 - val_accuracy: 0.5640

Epoch 00004: val_accuracy improved from 0.53683 to 0.56400, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_2.h5
Epoch 5/30
1436/1436 - 138s - loss: 0.7629 - accuracy: 0.7560 - val_loss: 1.5257 - val_accurac

225/225 [==============================] - 17s 77ms/step - loss: 1.7709 - accuracy: 0.6225
Found 45934 validated image filenames belonging to 7 classes.
Found 11484 validated image filenames belonging to 7 classes.
175
Epoch 1/30
1436/1436 - 142s - loss: 2.2950 - accuracy: 0.3495 - val_loss: 1.5337 - val_accuracy: 0.4545

Epoch 00001: val_accuracy improved from -inf to 0.45446, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_3.h5
Epoch 2/30
1436/1436 - 138s - loss: 1.7356 - accuracy: 0.4768 - val_loss: 1.3068 - val_accuracy: 0.5203

Epoch 00002: val_accuracy improved from 0.45446 to 0.52029, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_3.h5
Epoch 3/30
1436/1436 - 138s - loss: 1.4053 - accuracy: 0.5674 - val_loss: 1.4091 - val_accuracy: 0.5253

Epoch 00003: val_accuracy improved from 0.52029 to 0.52525, saving model to

225/225 [==============================] - 16s 73ms/step - loss: 1.9441 - accuracy: 0.6252
Found 45935 validated image filenames belonging to 7 classes.
Found 11483 validated image filenames belonging to 7 classes.
175
Epoch 1/30
1436/1436 - 143s - loss: 2.2980 - accuracy: 0.3457 - val_loss: 1.7288 - val_accuracy: 0.3727

Epoch 00001: val_accuracy improved from -inf to 0.37272, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 2/30
1436/1436 - 138s - loss: 1.7435 - accuracy: 0.4728 - val_loss: 1.6542 - val_accuracy: 0.4242

Epoch 00002: val_accuracy improved from 0.37272 to 0.42419, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_4.h5
Epoch 3/30
1436/1436 - 139s - loss: 1.4061 - accuracy: 0.5664 - val_loss: 1.7421 - val_accuracy: 0.4388

Epoch 00003: val_accuracy improved from 0.42419 to 0.43882, saving model to

225/225 [==============================] - 17s 76ms/step - loss: 2.2815 - accuracy: 0.6159
Found 45935 validated image filenames belonging to 7 classes.
Found 11483 validated image filenames belonging to 7 classes.
175
Epoch 1/30
1436/1436 - 143s - loss: 2.2877 - accuracy: 0.3490 - val_loss: 1.6209 - val_accuracy: 0.4218

Epoch 00001: val_accuracy improved from -inf to 0.42175, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_5.h5
Epoch 2/30
1436/1436 - 139s - loss: 1.7261 - accuracy: 0.4753 - val_loss: 1.4338 - val_accuracy: 0.4858

Epoch 00002: val_accuracy improved from 0.42175 to 0.48585, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_resnet_50_30epochs/transfer_learning_model_resnet_50_model_5.h5
Epoch 3/30
1436/1436 - 139s - loss: 1.4002 - accuracy: 0.5682 - val_loss: 1.3102 - val_accuracy: 0.5302

Epoch 00003: val_accuracy improved from 0.48585 to 0.53018, saving model to

225/225 [==============================] - 19s 83ms/step - loss: 2.1024 - accuracy: 0.6244


In [20]:
def transfer_learning_model_vgg_16():

  IMG_SHAPE = (224,224,3,)
 
  inputs = tf.keras.Input(shape=IMG_SHAPE)
  #inputs = tf.image.grayscale_to_rgb(y)
 
  x = tf.keras.applications.vgg16.preprocess_input(inputs)
  base_model = tf.keras.applications.vgg16.VGG16(input_shape=(224,224,3,),
                                               include_top=False,
                                               weights='imagenet')
  print(len(base_model.layers))
  base_model.trainable = True

  fine_tune_at = 13
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

  x = base_model(x)
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.5)(x)
  prediction_layer = tf.keras.layers.Dense(7, activation = "softmax")
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  #print(len(model.trainable_variables))
  return model

transfer_model_vgg_16 = transfer_learning_model_vgg_16()

58900480/58889256 [==============================] - 0s 0us/step
19


In [21]:
transfer_model_vgg_16.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 7)                 3591  

In [22]:
model_test(5, 30, transfer_learning_model_vgg_16, (224,224,), 'rgb')

Found 45934 validated image filenames belonging to 7 classes.
Found 11484 validated image filenames belonging to 7 classes.
19
Epoch 1/30
1436/1436 - 143s - loss: 2.7452 - accuracy: 0.1907 - val_loss: 1.9831 - val_accuracy: 0.2341

Epoch 00001: val_accuracy improved from -inf to 0.23406, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 2/30
1436/1436 - 135s - loss: 2.3799 - accuracy: 0.2620 - val_loss: 1.8638 - val_accuracy: 0.3272

Epoch 00002: val_accuracy improved from 0.23406 to 0.32715, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_1.h5
Epoch 3/30
1436/1436 - 135s - loss: 2.1258 - accuracy: 0.3464 - val_loss: 1.7702 - val_accuracy: 0.3758

Epoch 00003: val_accuracy improved from 0.32715 to 0.37583, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16

225/225 [==============================] - 19s 82ms/step - loss: 2.2733 - accuracy: 0.5942
Found 45934 validated image filenames belonging to 7 classes.
Found 11484 validated image filenames belonging to 7 classes.
19
Epoch 1/30
1436/1436 - 136s - loss: 2.7174 - accuracy: 0.1789 - val_loss: 2.0073 - val_accuracy: 0.2183

Epoch 00001: val_accuracy improved from -inf to 0.21830, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 2/30
1436/1436 - 135s - loss: 2.3479 - accuracy: 0.2676 - val_loss: 1.7253 - val_accuracy: 0.3330

Epoch 00002: val_accuracy improved from 0.21830 to 0.33299, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_2.h5
Epoch 3/30
1436/1436 - 135s - loss: 2.0759 - accuracy: 0.3615 - val_loss: 1.7578 - val_accuracy: 0.3699

Epoch 00003: val_accuracy improved from 0.33299 to 0.36991, saving model to /kaggle/work

225/225 [==============================] - 17s 77ms/step - loss: 2.3263 - accuracy: 0.6013
Found 45934 validated image filenames belonging to 7 classes.
Found 11484 validated image filenames belonging to 7 classes.
19
Epoch 1/30
1436/1436 - 136s - loss: 2.7638 - accuracy: 0.1845 - val_loss: 2.2747 - val_accuracy: 0.1860

Epoch 00001: val_accuracy improved from -inf to 0.18600, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_3.h5
Epoch 2/30
1436/1436 - 135s - loss: 2.3928 - accuracy: 0.2596 - val_loss: 1.9790 - val_accuracy: 0.2754

Epoch 00002: val_accuracy improved from 0.18600 to 0.27543, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_3.h5
Epoch 3/30
1436/1436 - 135s - loss: 2.1330 - accuracy: 0.3495 - val_loss: 1.8006 - val_accuracy: 0.3800

Epoch 00003: val_accuracy improved from 0.27543 to 0.38001, saving model to /kaggle/work

225/225 [==============================] - 20s 87ms/step - loss: 2.7968 - accuracy: 0.5809
Found 45935 validated image filenames belonging to 7 classes.
Found 11483 validated image filenames belonging to 7 classes.
19
Epoch 1/30
1436/1436 - 140s - loss: 2.7598 - accuracy: 0.1840 - val_loss: 2.0597 - val_accuracy: 0.2183

Epoch 00001: val_accuracy improved from -inf to 0.21832, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_4.h5
Epoch 2/30
1436/1436 - 135s - loss: 2.4015 - accuracy: 0.2509 - val_loss: 1.7626 - val_accuracy: 0.3532

Epoch 00002: val_accuracy improved from 0.21832 to 0.35322, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_4.h5
Epoch 3/30
1436/1436 - 135s - loss: 2.1512 - accuracy: 0.3416 - val_loss: 1.7993 - val_accuracy: 0.3716

Epoch 00003: val_accuracy improved from 0.35322 to 0.37159, saving model to /kaggle/work

225/225 [==============================] - 18s 80ms/step - loss: 1.8180 - accuracy: 0.5940
Found 45935 validated image filenames belonging to 7 classes.
Found 11483 validated image filenames belonging to 7 classes.
19
Epoch 1/30
1436/1436 - 136s - loss: 2.7599 - accuracy: 0.1882 - val_loss: 2.0453 - val_accuracy: 0.2381

Epoch 00001: val_accuracy improved from -inf to 0.23809, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_5.h5
Epoch 2/30
1436/1436 - 135s - loss: 2.3673 - accuracy: 0.2762 - val_loss: 1.7314 - val_accuracy: 0.3625

Epoch 00002: val_accuracy improved from 0.23809 to 0.36254, saving model to /kaggle/working/Saved_CNN_Models/transfer_learning_model_vgg_16_30epochs/transfer_learning_model_vgg_16_model_5.h5
Epoch 3/30
1436/1436 - 135s - loss: 2.1356 - accuracy: 0.3565 - val_loss: 1.7159 - val_accuracy: 0.3798

Epoch 00003: val_accuracy improved from 0.36254 to 0.37978, saving model to /kaggle/work

225/225 [==============================] - 18s 78ms/step - loss: 2.5049 - accuracy: 0.5843
